#从感知器到人工神经网络

在*第8章，感知器*里，我们介绍了感知器，一种线性模型用来做二元分类。感知器不是一个通用函数近似器；它的决策边界必须是一个超平面。上一章里面介绍的支持向量机，用核函数修正了感知器的不足，将特征向量有效的映射到更高维的空间使得样本成为线性可分的数据集。本章，我们将介绍人工神经网络（artificial neural networks，ANN），一种用于强大的非线性回归和分类模型，用新的策略来克服感知器的缺点。

<!-- TEASER_END-->

如果把感知器比喻成一个神经元，那么人工神经网络，即神经网，就是一个大脑。人脑就是由十几亿的神经元和上千亿的突触组成，人工神经网络是一种感知器或其他人工神经的有向图。这个图的边带有权重，这些权重是模型需要学习的参数。

有许多著作整本书描述人工神经网络；本章主要是对它的结构和训练方法进行介绍。目前scikit-learn的版本是0.16.1，有一些神经网络的模型已经被开发出来，但是并没有合并到正式版。